In [1]:
%matplotlib inline
from ipywidgets import interactive 
import seaborn as sb
import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
#Define the matrix
#write better documentation
def E_Matrix(r, theta):
    #Input conditions of problem
    Q = 1
    G = 1
    k = 1
    w = 1
    t = 0
    C = 4*G*Q*(k**2)
    Err = ((-C/r**3)*((3/((k*r)**2)-1)*np.cos(k*r-w*t)+3*np.sin(k*r-w*t)/(k*r)))*(3*np.cos(theta)**2-1)
    Ert = (-C/r**3)*((6/((k*r)**2)-3)*np.cos(k*r-w*t)-(k*r-6/(k*r))*np.sin(k*r-w*t))*(np.sin(theta)*np.cos(theta))
    Edif = (-C/(2*r**3))*((-3/((k*r)**2)+3-(k*r)**2)*np.cos(k*r-w*t)-(3/(k*r)-2*k*r)*np.sin(k*r-w*t))*(np.sin(theta)**2)
    Matrix = np.array([[Err, Ert], 
                       [Ert, (Edif-Err)/2]])
    
    return Matrix

In [3]:
## Use the x and y coordinates, create a grid, convert those into polar and get the matrix values at each point

nx, ny = 100,200 
grid = 10
       
XMAX, YMAX = 8, 8
x = np.linspace(0, XMAX, nx)
y = np.linspace(-YMAX, YMAX, ny)
X, Y = np.meshgrid(x, y)
r, theta = np.hypot(X,Y), np.arctan2(X,Y)
arrayR = ([X/r, Y/r], [-Y/r, X/r])
Matrix1 = E_Matrix(r, theta)

#after getting matrix values, swap axes for iteration 
MatrixSwap1 = np.swapaxes(np.swapaxes(Matrix1,0,2),1,3)
eigval1, eigvec1 = np.linalg.eig(MatrixSwap1)


Rotation1 = [[np.sin(theta), np.cos(theta)],
            [np.cos(theta), -np.sin(theta)]]

RotationSwap1 = np.swapaxes(np.swapaxes(Rotation1,0,2),1,3)

cartMatrix1 = np.matmul(RotationSwap1, eigvec1)

#Separate into larger and smaller magnitude eigenvalues
large_eigval1 = np.amax(np.abs(eigval1), axis=2)
small_eigval1 = np.amin(np.abs(eigval1), axis=2)

In [4]:
#Create a scaled version of the plot where the eigenvector corresponding to the larger eigenvalue is scaled to 1
#and the eigenvector corresponding to the smaller eigenvalue is scaled to 1*ratio of the eigenvalues. 
for i in range(0, ny):
    for j in range(0, nx):
        if abs(eigval1[i,j,0]) > abs(eigval1[i,j,1]):
            cartMatrix1[i,j,0,0] = cartMatrix1[i,j,0,0]
            cartMatrix1[i,j,1,0] = cartMatrix1[i,j,1,0]
            cartMatrix1[i,j,0,1] = cartMatrix1[i,j,0,1]*abs(eigval1[i,j,1]/eigval1[i,j,0])
            cartMatrix1[i,j,1,1] = cartMatrix1[i,j,1,1]*abs(eigval1[i,j,1]/eigval1[i,j,0])
        else:
            cartMatrix1[i,j,0,1] = cartMatrix1[i,j,0,1]
            cartMatrix1[i,j,1,1] = cartMatrix1[i,j,1,1]
            cartMatrix1[i,j,0,0] = cartMatrix1[i,j,0,0]*abs(eigval1[i,j,0]/eigval1[i,j,1])
            cartMatrix1[i,j,1,0] = cartMatrix1[i,j,1,0]*abs(eigval1[i,j,0]/eigval1[i,j,1])

In [5]:
colorVec = np.array([])

for i in range(0, ny):
    for j in range(0, nx):
        if abs(eigval1[i,j,0]) > abs(eigval1[i,j,1]):
            if np.sign(eigval1[i,j,0]) > 0:
                cartMatrix1[i,j,:,0] = colorVec[i,j,:,0]
                cartMatrix1[i,j,:,1] = colorVec[i,j,:,1]
            elif np.sign(eigval1[i,j,0]) < 0:
                cartMatrix1[i,j,:,0] = colorVec[i,j,:,1]
                cartMatrix1[i,j,:,1] = colorVec[i,j,:,0]
        elif abs(eigval1[i,j,1]) > abs(eigval1[i,j,0]):
            if np.sign(eigval1[i,j,1]) > 0:
                cartMatrix1[i,j,:,1] = colorVec[i,j,:,0]
                cartMatrix1[i,j,:,0] = colorVec[i,j,:,1]
            elif np.sign(eigval1[i,j,1]) < 0:
                cartMatrix1[i,j,:,0] = colorVec[i,j,:,1]
                cartMatrix1[i,j,:,1] = colorVec[i,j,:,0]

IndexError: too many indices for array

In [ ]:
##### plot of eigenvectors for both eigenvalues + and - 
#this is the same thing as below but without the scaling related to eigenvalue magnitude
fig, ax1 = plt.subplots()
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],colorVec[::grid,::grid,0,0],colorVec[::grid,::grid,1,0], units='xy', pivot='mid', color='blue', headlength=0, headwidth=1, scale=1)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],colorVec[::grid,::grid,0,1],colorVec[::grid,::grid,1,1], units='xy', pivot='mid', color='red', headlength=0, headwidth=1, scale=1)
z1 = large_eigval1
ax1 = plt.imshow(np.log10(np.abs(z1)), interpolation='bilinear', cmap='Greys', extent = [-.5,XMAX,-YMAX,YMAX])
plt.colorbar()
plt.show()

In [ ]:
##### plot of eigenvectors for both eigenvalues + and - 
#this is the same thing as below but without the scaling related to eigenvalue magnitude
fig, ax1 = plt.subplots()
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],cartMatrix1[::grid,::grid,0,0],cartMatrix1[::grid,::grid,1,0], units='xy', pivot='mid', headlength=0, headwidth=1, scale=2)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],cartMatrix1[::grid,::grid,0,1],cartMatrix1[::grid,::grid,1,1], units='xy', pivot='mid', headlength=0, headwidth=1, scale=2)
z1 = large_eigval1
ax1 = plt.imshow(np.log10(np.abs(z1)), interpolation='bilinear', cmap='Greys', extent = [-.5,XMAX,-YMAX,YMAX])
plt.colorbar()
plt.show()

In [ ]:
#This was an attempt at doing the eigvector calculation myself and seeing if i could match the results from
#numerical method. It did not work but is kept here for historical purposes.
#
#matrix[0,0]=matrix[0,0:,:]


#Define terms of quadratic equation
a = 1
b = -(Matrix[0,0] + Matrix[1,1])
c = Matrix[0,0]*Matrix[1,1] - Matrix[0,1]*Matrix[1,0]

#Calculate eigenvalues 
Eigvalue1 = (-b + np.sqrt(b**2-4*a*c))/2*a
Eigvalue2 = (-b - np.sqrt(b**2-4*a*c))/2*a

#Calculate the Eigenvectors
I = np.identity(nx)
print(np.shape(MatrixSwap))
EigMatrix1 = Matrix - Eigvalue1*I
EigMatrix2 = Matrix - Eigvalue2*I
normalize1 = np.sqrt(EigMatrix1[0,0]**2 + EigMatrix1[1,0]**2)
normalize2 = np.sqrt(EigMatrix2[0,0]**2 + EigMatrix2[1,0]**2)
EigVec = np.array([[EigMatrix1[0,0]/normalize1, -EigMatrix2[0,0]/normalize2],
                    [EigMatrix1[1,0]/normalize1, -EigMatrix2[1,0]/normalize2]])
EigVecSwap = np.swapaxes(np.swapaxes(EigVec,0,2),1,3)
CartAnalytical = np.matmul(RotationSwap,EigVecSwap)

fig, ax1 = plt.subplots()
ax1.quiver(X[::grid,::grid],Y[::grid,::grid], EigVecSwap[::grid,::grid,0,0],EigVecSwap[::grid,::grid,1,0], units="xy", headwidth=0, scale=1)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],EigVecSwap[::grid,::grid,0,1],EigVecSwap[::grid,::grid,1,1], units="xy", headwidth=0, scale=1)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],-EigVecSwap[::grid,::grid,0,0],-EigVecSwap[::grid,::grid,1,0], units="xy", headwidth=0, scale=1)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],-EigVecSwap[::grid,::grid,0,1],-EigVecSwap[::grid,::grid,1,1], units="xy", headwidth=0, scale=1)
plt.show()

In [ ]:
#plot of heatmap for positive eigenvalues
#plot of eigenvectors for both eigenvalues + and - 
fig, ax1 = plt.subplots()
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],cartMatrix[::grid,::grid,0,0],cartMatrix[::grid,::grid,1,0], units="xy", headwidth=0, scale=1)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],cartMatrix[::grid,::grid,0,1],cartMatrix[::grid,::grid,1,1], units="xy", headwidth=0, scale=1)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],-cartMatrix[::grid,::grid,0,0],-cartMatrix[::grid,::grid,1,0], units="xy", headwidth=0, scale=1)
ax1.quiver(X[::grid,::grid],Y[::grid,::grid],-cartMatrix[::grid,::grid,0,1],-cartMatrix[::grid,::grid,1,1], units="xy", headwidth=0, scale=1)
z2 = small_eigval
ax1 = plt.imshow(np.log(np.abs(z2)), norm=LogNorm(), interpolation='bilinear', extent = [-7,7,-7,7])
plt.colorbar()
plt.show()

In [ ]:
#Make for loop to calculate each eigenvalue separately for each of the nxn matrices. ?? i think that makes sense
#eigval = np.zeros((nx, ny, 2))
#eigvec = np.zeros((nx,ny, 2, 2))
#calc1 = np.zeros((nx, ny, 2, 2))
#calc2 = np.zeros((nx, ny, 2, 2))
#for i in range(nx):
#    for j in range(ny):
#        a, b = np.linalg.eig(Matrix[:,:,i,j])
#        eigval[i,j,0] = a[0]
#        eigval[i,j,1] = a[1]
#        eigvec[i,j,0,0] = b[0,0]
#        eigvec[i,j,1,0] = b[1,0]
#        eigvec[i,j,0,1] = b[0,1]
#        eigvec[i,j,0,1] = b[1,1]
#        calc1[i,j,0,0] = a[0]*b[0,0]
#        calc1[i,j,1,0] = a[0]*b[1,0]
#        calc1[i,j,0,1] = a[1]*b[0,1]
#        calc1[i,j,1,1] = a[1]*b[1,1]
#        calc2[i,j,0,0] = Matrix[0,0,i,j]*b[0,0] + Matrix[0,1,i,j]*b[1,0]
#        calc2[i,j,1,0] = Matrix[1,0,i,j]*b[0,0] + Matrix[1,1,i,j]*b[1,0]
#        calc2[i,j,0,1] = Matrix[0,0,i,j]*b[0,1] + Matrix[0,1,i,j]*b[0,1]
#        calc2[i,j,1,1] = Matrix[1,0,i,j]*b[0,1] + Matrix[1,1,i,j]*b[1,1]